#**АИС. Лабораторная работа №1**

**Максимов Антон, 17ПМИ**

In [0]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt

In [60]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [4]:
cd /content/gdrive/'My Drive'/АИО/benchmarks/

/content/gdrive/My Drive/АИО/benchmarks


In [0]:
a=!ls -1  *.*
bad_t = []
bad_w = []
good_t = []
good_w = []
for i in range(0,4):
  with open(a[i]) as f:
    bad_t.append(f.read())
for i in range(4,8):
  with open(a[i]) as f:
    bad_w.append(f.read())
for i in range(8,12):
  with open(a[i]) as f:
    good_t.append(f.read())
for i in range(12,16):
  with open(a[i]) as f:
    good_w.append(f.read())          

In [0]:
print(bad_t)
print(bad_w)
print(good_t)
print(good_w)

##**Наивный алгоритм**

In [0]:
def Naive(string, pattern):
    counter = 0
    N = len(string)
    M = len(pattern)
    index = []
    for i in range(N - M + 1):
      for j in range(M):
        counter+=1
        if string[i + j] != pattern[j]:
          break
        counter+=1  
        if j == (M - 1):  
          index.append(i)
          return index, counter 
    return index, counter 

In [370]:
Naive(good_t[0], good_w[0])

([599], 667)

##**Алгоритм Рабина-Карпа**

In [0]:
def RabinKarp(string, pattern, prime = 9901, base = 256):
    N = len(string)
    M = len(pattern)
    counter = 0
    hash_pattern = 0
    hash_string = 0
    h = pow(base, M-1) % prime
    index = []
    for i in range(M):
      hash_pattern = (base * hash_pattern + ord(pattern[i])) % prime
      hash_string = (base * hash_string + ord(string[i])) % prime

    for i in range(N - M + 1):
      counter+= 1
      if hash_pattern == hash_string:
        counter+= 1
        if string[i:i + M] == pattern:
          index.append(i)
          return index, counter
      counter+= 1    
      if i < N - M:
        hash_string = (base * (hash_string - ord(string[i]) * h) + ord(string[i+M])) % prime 
    return index, counter      

In [360]:
RabinKarp(good_t[0], good_w[0])

([599], 1200)

##**Алгоритм Бойера-Мура-Хорспула**

In [0]:
def build_table(pattern):
  table = {}
  n = len(pattern)
  for i in range(n - 1):
    table[pattern[i]] = n - i - 1
  return table

In [0]:
def BMX(string, pattern):
  counter = 0
  table = build_table(pattern)
  N = len(string)
  M = len(pattern)
  i = M - 1
  index = []

  while i < N and len(index)==0:
    counter+=1
    j = 0
    while j < M:
      counter+=1
      if string[i - j] != pattern[M - j -1]:
        break
      j+=1
    counter+=1  
    if j == M:
      index.append(i - j + 1)
     
    elif j!=0:
      counter+=1 
      i+=1
    else:
      shift = table.get(string[i - j], M) #или сдвиг по таблице или по длине паттерна
      i += shift  

  return index , counter     

In [368]:
BMX(good_t[0], good_w[0])

([599], 186)

##**Алгоритм Кнутта-Мориса-Пратта**

In [0]:
def FoundPrefix(string):
  counter=0
  n = len(string)
  pi = [0] * n
  for i in range(1, n):
    k = pi[i - 1]
    while k > 0 and string[k] != string[i]:
      counter+=1 
      k = pi[k - 1]
    counter+=1   
    if string[k] == string[i]:
      k += 1
    pi[i] = k  
  return pi, counter

def KMP(string, pattern):
  index = []
  c =0
  k = 0
  n = len(pattern)
  F, counter = FoundPrefix(pattern)
  c+=counter
  for i in range(0, len(string)):
    while k > 0 and pattern[k] != string[i]:
      c+=1
      k = F[k-1]
    c+=1  
    if pattern[k] == string[i]:
      k += 1
    c+=1  
    if k == n:
      index.append(i - k + 1)
      return index,c
      k = F[k - 1]
  return index,c

In [365]:
KMP(good_t[0], good_w[0])

([599], 1266)

##**Тестирование**

In [0]:
import timeit

**Будем прогонять алгоритмы по каждому примеру 100 раз, замерим общее время**

**Далее повторим это 100 раз и вычислим среднее, то есть получим общее среднее выполнение алгоритма 10000 раз**

###**good**

In [0]:
good_Naive = []
for test in range(0, 4):
  good_Naive.append(np.mean(timeit.repeat("Naive(good_t[{0}], good_w[{0}])".format(test), setup="from __main__ import Naive, good_t, good_w", repeat = 100, number = 100)))

In [0]:
good_RabinKarp = []
for test in range(0, 4):
  good_RabinKarp.append(np.mean(timeit.repeat("RabinKarp(good_t[{0}], good_w[{0}])".format(test), setup="from __main__ import RabinKarp, good_t, good_w", repeat = 100, number = 100)))

In [0]:
good_BMX = []
for test in range(0, 4):
  good_BMX.append(np.mean(timeit.repeat("BMX(good_t[{0}], good_w[{0}])".format(test), setup="from __main__ import BMX, good_t, good_w", repeat = 100, number = 100)))

In [0]:
good_KMP = []
for test in range(0, 4):
  good_KMP.append(np.mean(timeit.repeat("KMP(good_t[{0}], good_w[{0}])".format(test), setup="from __main__ import KMP, good_t, good_w", repeat = 100, number = 100)))

###**bad**

In [0]:
bad_Naive = []
for test in range(0, 4):
  bad_Naive.append(np.mean(timeit.repeat("Naive(bad_t[{0}], bad_w[{0}])".format(test), setup="from __main__ import Naive, bad_t, bad_w", repeat = 100, number = 100)))

In [0]:
bad_RabinKarp = []
for test in range(0, 4):
  bad_RabinKarp.append(np.mean(timeit.repeat("RabinKarp(bad_t[{0}], bad_w[{0}])".format(test), setup="from __main__ import RabinKarp, bad_t, bad_w", repeat = 100, number = 100)))

In [0]:
bad_BMX = []
for test in range(0, 4):
  bad_BMX.append(np.mean(timeit.repeat("BMX(bad_t[{0}], bad_w[{0}])".format(test), setup="from __main__ import BMX, bad_t, bad_w", repeat = 100, number = 100)))

In [0]:
bad_KMP = []
for test in range(0, 4):
  bad_KMP.append(np.mean(timeit.repeat("KMP(bad_t[{0}], bad_w[{0}])".format(test), setup="from __main__ import KMP, bad_t, bad_w", repeat = 100, number = 100)))

**Общая таблица**

In [336]:
df = pd.DataFrame({
'Алгоритм': ['Наивный', 'Рабина-Карпа', 'Бойера-Мура-Хорспула', 'Кнута-Морриса-Пратта'],
'good_1': [good_Naive[0], good_RabinKarp[0], good_BMX[0], good_KMP[0]],
'good_2': [good_Naive[1], good_RabinKarp[1], good_BMX[1], good_KMP[1]],
'good_3': [good_Naive[2], good_RabinKarp[2], good_BMX[2], good_KMP[2]],
'good_4': [good_Naive[3], good_RabinKarp[3], good_BMX[3], good_KMP[3]],
'bad_1': [bad_Naive[0], bad_RabinKarp[0], bad_BMX[0], bad_KMP[0]],
'bad_2': [bad_Naive[1], bad_RabinKarp[1], bad_BMX[1], bad_KMP[1]],
'bad_3': [bad_Naive[2], bad_RabinKarp[2], bad_BMX[2], bad_KMP[2]],
'bad_4': [bad_Naive[3], bad_RabinKarp[3], bad_BMX[3], bad_KMP[3]],
'Среднее время по всем тестам': [np.mean(good_Naive+bad_Naive), np.mean(good_RabinKarp+bad_RabinKarp),np.mean(good_BMX+bad_BMX), np.mean(good_KMP+bad_KMP)]
})
df

,Алгоритм,good_1,good_2,good_3,good_4,bad_1,bad_2,bad_3,bad_4,Среднее время по всем тестам
0,Наивный,0.022607,0.023079,0.076625,0.351287,0.000483,0.011852,1.089112,56.853518,7.303570
1,Рабина-Карпа,0.026897,0.033302,0.102221,0.512135,0.000553,0.003664,0.035869,0.179306,0.111743
2,Бойера-Мура-Хорспула,0.003569,0.003917,0.015689,0.024040,0.000533,0.003812,0.040024,0.207119,0.037338
3,Кнута-Морриса-Пратта,0.011423,0.014153,0.048324,0.172341,0.000449,0.003205,0.032170,0.187175,0.058655


**Количество операций сравнения**

In [0]:
good_Naive = []
good_RabinKarp = []
good_BMX = []
good_KMP = []
bad_Naive = []
bad_RabinKarp = []
bad_BMX = []
bad_KMP = []

for i in range(0, 4):
  good_Naive.append(Naive(good_t[i], good_w[i])[1])
  bad_Naive.append(Naive(bad_t[i], bad_w[i])[1])
  good_RabinKarp.append(RabinKarp(good_t[i], good_w[i])[1])
  bad_RabinKarp.append(RabinKarp(bad_t[i], bad_w[i])[1])
  good_BMX.append(BMX(good_t[i], good_w[i])[1])
  bad_BMX.append(BMX(bad_t[i], bad_w[i])[1])
  good_KMP.append(KMP(good_t[i], good_w[i])[1])
  bad_KMP.append(KMP(bad_t[i], bad_w[i])[1])

In [372]:
df = pd.DataFrame({
'Алгоритм': ['Наивный', 'Рабина-Карпа', 'Бойера-Мура-Хорспула', 'Кнута-Морриса-Пратта'],
'good_1': [good_Naive[0], good_RabinKarp[0], good_BMX[0], good_KMP[0]],
'good_2': [good_Naive[1], good_RabinKarp[1], good_BMX[1], good_KMP[1]],
'good_3': [good_Naive[2], good_RabinKarp[2], good_BMX[2], good_KMP[2]],
'good_4': [good_Naive[3], good_RabinKarp[3], good_BMX[3], good_KMP[3]],
'bad_1': [bad_Naive[0], bad_RabinKarp[0], bad_BMX[0], bad_KMP[0]],
'bad_2': [bad_Naive[1], bad_RabinKarp[1], bad_BMX[1], bad_KMP[1]],
'bad_3': [bad_Naive[2], bad_RabinKarp[2], bad_BMX[2], bad_KMP[2]],
'bad_4': [bad_Naive[3], bad_RabinKarp[3], bad_BMX[3], bad_KMP[3]],
'Среднее кол-во операций по всем тестам': [np.mean(good_Naive+bad_Naive), np.mean(good_RabinKarp+bad_RabinKarp),np.mean(good_BMX+bad_BMX), np.mean(good_KMP+bad_KMP)]
})
df

,Алгоритм,good_1,good_2,good_3,good_4,bad_1,bad_2,bad_3,bad_4,Среднее кол-во операций по всем тестам
0,Наивный,667,780,2519,9706,28,1730,179300,7998000,1024091.25
1,Рабина-Карпа,1200,1222,3260,19048,18,182,1802,8002,4341.75
2,Бойера-Мура-Хорспула,186,148,484,1282,28,282,2802,13002,2276.75
3,Кнута-Морриса-Пратта,1266,1474,4469,19319,29,307,3097,15997,5744.75


<center>

![Alt Text](https://i.gifer.com/VEFN.gif)
</center>
